In [135]:
from flask import Flask
import redis
from kafka import KafkaConsumer
from pykafka import KafkaClient
from datetime import timedelta,datetime
from flask import request,render_template

In [136]:
r = redis.Redis(host='localhost', port=6379, db=0)

In [137]:
#part1
def part1():
    channel_key = 'tasnimna'
    count = 0
    for i in range(6):
        last_hour_date_time = datetime.now() - timedelta(hours = i)
        last_time = last_hour_date_time.strftime("%m") + last_hour_date_time.strftime("%d") + last_hour_date_time.strftime("%H")
        if(r.hexists(channel_key+last_time,last_time)):
            count += int(r.hget(channel_key+last_time,last_time))
    return 'number of post from channel '+channel_key+' in 6 hour ago is ' + str(count)
    

In [138]:
part1()

'number of post from channel tasnimna in 6 hour ago is 0'

In [139]:
#part2
def part2():
    count = 0 
    for i in range(24):
        last_hour_date_time = datetime.now() - timedelta(hours = i)
        last_time = last_hour_date_time.strftime("%m") + last_hour_date_time.strftime("%d") + last_hour_date_time.strftime("%H")
        if(r.hexists('count'+last_time,last_time)):
            count += int(r.hget('count'+last_time,last_time))
    return 'total number of posts in 24 hours ago is ' + str(count)

In [140]:
part2()

'total number of posts in 24 hours ago is 42'

In [142]:
#part3
def part3():
    date_time = datetime.now()
    nowtime = date_time.strftime("%m") + date_time.strftime("%d") + date_time.strftime("%H")
    last_hour_date_time = date_time - timedelta(hours = 1)
    last_time = last_hour_date_time.strftime("%m") + last_hour_date_time.strftime("%d") + last_hour_date_time.strftime("%H")

    hashtags = [i.decode("utf-8")[:-8] for i in r.hgetall('hashtags'+nowtime)] +[i.decode("utf-8")[:-8] for i in r.hgetall('hashtags'+last_time)]

    hashtags = set(hashtags)
    hashtags = list(hashtags)
#     print(hashtags)
    count = {}
    for hashtag in hashtags:
        c = 0 
        for i in range(60):    
            last_min_date_time = date_time - timedelta(minutes = i)
            last_time_min = last_min_date_time.strftime("%m") + last_min_date_time.strftime("%d") + last_min_date_time.strftime("%H") + last_min_date_time.strftime("%m")

            if(r.hexists('hashtags'+last_time,hashtag+last_time_min) ):
                c += int(r.hget('hashtags'+last_time,hashtag+last_time_min))
            if(r.hexists('hashtags'+ nowtime,hashtag+last_time_min) ):
                c += int(r.hget('hashtags'+nowtime,hashtag+last_time_min))
        count[hashtag] = c
    return 'count of each hashtags in 60 min ago : ' + str(count)

In [143]:
part3()

'count of each hashtags in 60 min ago : {}'

In [144]:
#part4
def part4():
    last_hash = [i.decode("utf-8") for i in r.lrange('hashtagList',0,999)]
    stringres = ""
    for i in range(len(last_hash)):
        
        stringres =  stringres + '|' + last_hash[i]
        if(i%10 == 9):
            stringres =  stringres + ' <br/> '
    return 'last 1000 hashtags : ' +' <br/> '+stringres

In [145]:
#part5
def part5():
    last_posts = [i.decode("utf-8") for i in r.lrange('postList',0,99)]
    stringres = ""
    for i in range(len(last_posts)):
         stringres =  stringres + str(i) + ')  ' + last_posts[i] +' <br/> '
        
    return 'last 100 post is :' +' <br/> ' + stringres

In [146]:
def result():
    n = ""
    for i in range(120):
        n = n + " - "
    res = part1() + ' <br/> '+n+ ' <br/> ' + part2() + ' <br/> '+n+ ' <br/> ' + part3() + ' <br/> '+n+ ' <br/> ' + part4() + ' <br/> '+n+ ' <br/> ' + part5() 
    return res

In [ ]:
app = Flask(__name__)
   
@app.route('/')
def resflask():
    return result()

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Aug/2020 20:08:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2020 20:11:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2020 20:28:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2020 21:09:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2020 01:22:43] "GET / HTTP/1.1" 200 -
